# Excitation Generation via KernelSynth

For a time series zero-shot forecasting model, high-quality training data is essential.
To further supplement the training dataset, [Chronos](https://arxiv.org/abs/2403.07815) proposed
KernelSynth, a method to generate synthetic time series using Gaussian processes (GPs). The model struction of Chronos are show as fellow:

<p align="center">
  <img src="https://raw.githubusercontent.com/amazon-science/chronos-forecasting/main/figures/main-figure.png" width="100%">
  <br />
  <span>
    High-level depiction of Chronos. (<b>Left</b>) The input time series is scaled and quantized to obtain a sequence of tokens. (<b>Center</b>) The tokens are fed into a language model which may either be an encoder-decoder or a decoder-only model. The model is trained using the cross-entropy loss. (<b>Right</b>) During inference, we autoregressively sample tokens from the model and map them back to numerical values. Multiple trajectories are sampled to obtain a predictive distribution. The original code link is: https://github.com/amazon-science/chronos-forecasting.
  </span>
</p>


KernelSynth is inspired by the Automatic Statistician (Duvenaud et al., 2013), where a compositional search over a space of GP kernels is performed to explain the structure of a time series.
Chronos use the inverse of this process randomly compose GP kernels to generate new time series. The data generation process is shown below:

<p align="center">
  <img src="https://raw.githubusercontent.com/wwhenxuan/S2Generator/master/images/algorithm_kernel_synth.jpg" width="100%">
</p>

The different kernel expressions and hyperparameter settings involved in the data generation process are shown in the following table:

| Kernel             | Formula                                                      | Hyperparameters                                              |
| ------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ |
| Constant           | $\mathcal{K}_{\mathrm{Const}}(x, x') = C$                    | $C=1$                                                        |
| White Noise        | $\mathcal{K}_{\mathrm{White}}(x, x') = \sigma_n \cdot \mathrm{1}_{(x=x')}$ | $\sigma_n \in \{ 0.1, 1 \}$                                  |
| Linear             | $\mathcal{K}_{\mathrm{Lin}}(x, x') = \sigma ^ 2 + x \cdot x'$ | $\sigma \in \{ 0, 1, 10 \}$                                  |
| RBF                | $\mathcal{K}_{\mathrm{RBF}}(x, x') = \mathrm{exp} \left ( - \frac{\left \| x - x' \right \| ^ 2}{2 l ^ 2} \right ) $ | $l \in {0.1, 1, 10}$                                         |
| Rational Quadratic | $\mathcal{K}_{\mathrm{RQ}}(x, x') = \left ( 1 + \frac{\left \| x - x' \right \| ^ 2}{2 \alpha} \right )$ | $\alpha \in \{ 0.1, 1, 10 \}$                                |
| Reriodic           | $\mathcal{K}_{\mathrm{Per}}(x, x') = \mathrm{exp} \left ( - 2 \sin ^ 2 \left ( \pi \frac{\left \| x - x' \right \|}{p} \right ) \right )$ | $p \in \left \{ 24, 48, 96, 168, 336, 672, 7, 14, 30, 60, 365, 730, 4, 26, 52, 6, 12, 40, 10 \right \}$ |